In [1]:
import numpy as np
from Source import RS64_Encoder
from Source import RS64_Decoder
from Source.GF64 import GFE

import keras
from keras.layers import Dense, Input,Conv1D, MaxPooling1D, Flatten, Embedding, Reshape, ReLU, Add, Concatenate, Average
from keras.models import Sequential, Model
from keras.models import load_model

Using TensorFlow backend.


In [2]:
m = 6
t_RS = 8
b_RS = 1
order_alpha=(2**m)-1
length_parity=2*t_RS
n_RS=order_alpha
k_RS=n_RS - length_parity

Z_RS=[]
for i in range(2*t_RS):    
    Z_RS.append((b_RS+i)%order_alpha)
    
g_x = RS64_Encoder.Generation_Polynomial(Z_RS, order_alpha)

In [3]:
codeword_size = 10**6
# codeword_RS = RS64_Encoder.Encoder(m, t_RS, b_RS, order_alpha, k_RS, Z_RS, g_x, codeword_size)

# cx = np.power(-1,GFE[(codeword_RS+1).astype(np.int)])
# cx = cx.reshape((codeword_size, -1))

In [4]:
cx = np.random.randint(2, size=(codeword_size, 378))
cx = np.where(cx > 0, -1, 1)
cx = cx.reshape((codeword_size, -1))

In [5]:
Eb_No = 3
rx = RS64_Decoder.Receive_Bit(Eb_No, cx, k_RS, n_RS)

In [6]:
#### model make####
input_layer = Input(shape=(378,))
line1_Dense1 = Dense(126, activation='elu')(input_layer)
line1_Dense2 = Dense(42, activation='elu')(line1_Dense1)
line1_Dense3 = Dense(126, activation='elu')(line1_Dense2)
line1_output_layer = Dense(378, activation='tanh')(line1_Dense1)

line2_Dense1 = Dense(189, activation='elu')(input_layer)
line2_Dense2 = Dense(95, activation='elu')(line2_Dense1)
line2_Dense3 = Dense(189, activation='elu')(line2_Dense2)
line2_output_layer = Dense(378, activation='tanh')(line2_Dense3)

line3_Dense1 = Dense(378, activation='elu')(input_layer)
line3_Dense2 = Dense(378, activation='elu')(line3_Dense1)         
line3_Dense3 = Dense(378, activation='elu')(line3_Dense2)
line3_output_layer = Dense(378, activation='tanh')(line3_Dense3)

line4_Dense1 = Dense(756, activation='elu')(input_layer)
line4_Dense2 = Dense(1512, activation='elu')(line4_Dense1)         
line4_Dense3 = Dense(756, activation='elu')(line4_Dense2)
line4_output_layer = Dense(378, activation='tanh')(line4_Dense3)

line5_Dense1 = Dense(1134, activation='elu')(input_layer)
line5_Dense2 = Dense(3402, activation='elu')(line5_Dense1)         
line5_Dense3 = Dense(1134, activation='elu')(line5_Dense2)
line5_output_layer = Dense(378, activation='tanh')(line5_Dense3)

add_layer = Add()([line1_output_layer, line2_output_layer, line3_output_layer, line4_output_layer, line5_output_layer])

output_layer = Dense(378, activation='sigmoid')(add_layer)

model = Model(inputs = input_layer, outputs = output_layer)

model.compile(optimizer='adam', loss='binary_cross_entropy', metrics=['accuracy'])

In [7]:
model.fit(rx[:int(0.7*codeword_size)], cx[:int(0.7*codeword_size)], 
          epochs=100,
          batch_size=10000,
          shuffle=True,
          validation_data=(rx[int(0.7*codeword_size):], cx[int(0.7*codeword_size):]))

700000/700000 [==============================] - 16s 23us/step - loss: 0.2490 - acc: 0.0056 - val_loss: 0.2648 - val_acc: 0.0065
Epoch 88/100
700000/700000 [==============================] - 16s 23us/step - loss: 0.2487 - acc: 0.0055 - val_loss: 0.2646 - val_acc: 0.0051
Epoch 89/100
700000/700000 [==============================] - 17s 24us/step - loss: 0.2485 - acc: 0.0055 - val_loss: 0.2645 - val_acc: 0.0053
Epoch 90/100
700000/700000 [==============================] - 16s 23us/step - loss: 0.2483 - acc: 0.0056 - val_loss: 0.2646 - val_acc: 0.0057
Epoch 91/100
700000/700000 [==============================] - 16s 23us/step - loss: 0.2481 - acc: 0.0056 - val_loss: 0.2647 - val_acc: 0.0055
Epoch 92/100
700000/700000 [==============================] - 16s 23us/step - loss: 0.2480 - acc: 0.0055 - val_loss: 0.2649 - val_acc: 0.0056
Epoch 93/100
700000/700000 [==============================] - 16s 23us/step - loss: 0.2479 - acc: 0.0055 - val_loss: 0.2648 - val_acc: 0.0051
Epoch 94/100
700000

In [8]:
model.save('model/model.h5')

In [ ]:
pred_vector_RS = model.predict(r_vector_RS)
print(pred_vector_RS)
pred_vector_RS = np.reshape(pred_vector_RS,[codeword_size, n_RS]) 
pred_vector_RS = np.rint(pred_vector_RS* order_alpha) - 1

In [ ]:
print(pred_vector_RS)

In [ ]:
syndrome_RS = []
syndrome_check = []
for i in range(codeword_size):
    s_RS, s_c = RS64_Decoder.Syndrome_Check(t_RS, pred_vector_RS[i], order_alpha, Z_RS)
    syndrome_RS.extend(s_RS)
    syndrome_check.append(s_c)
syndrome_RS = np.reshape(syndrome_RS, [codeword_size, 2*t_RS])

In [ ]:
Estimated_Codeword = []
for i in range(codeword_size):
    E_Codeword = RS64_Decoder.Decoder(syndrome_RS[i], syndrome_check[i], pred_vector_RS[i], t_RS, order_alpha, m)
    Estimated_Codeword.extend(E_Codeword)
Estimated_Codeword = np.reshape(Estimated_Codeword, [codeword_size, n_RS])

In [ ]:
Estimated_Message=Estimated_Codeword[:,length_parity:]
msg_RS = codeword_RS[:,length_parity:]

Symbol_error_count=0
for i  in range(codeword_size):
    for j in range(k_RS):
        if Estimated_Message[i][j] != msg_RS[i][j]:
            Symbol_error_count+=1
    
SER=Symbol_error_count/(codeword_size*k_RS)

print("Symbol Error Rate:",SER)